# Algebraic operations on complex numbers

Import necessary libraries

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool

In [ ]:
import cmath
import sympy as sp

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0, model = "gpt-3.5-turbo")

Define necassary functions for arithmetic operations

In [ ]:
def add_complex(a: complex, b: complex) -> complex:
    return a + b

def subtract_complex(a: complex, b: complex) -> complex:
    return a - b

def multiply_complex(a: complex, b: complex) -> complex:
    return a * b

def divide_complex(a: complex, b:complex) -> complex:
    if b == 0:
        return "Division by zero is undefined."
    return a / b

In [ ]:
def solve_complex_equation(equation: str) -> str:
    z = sp.symbols('z')
    try:
        # Convert string equation to a sympy equation
        sympy_eq = sp.simplify(equation)
        print(sympy_eq)
        # Solve for z
        solutions = sp.solve(sympy_eq, z)
        return f"Solutions: {solutions}"
    except Exception as e:
        return str(e)

In [ ]:
def nth_root_complex(a: complex, n: int) -> list:
    return [cmath.exp(cmath.log(a)/n + 2j * cmath.pi * k/n) for k in range(n)]

One function to access all of the above functions (one function since I will use it in Tool)

In [ ]:
def perform_complex_operation(operation: str, *args):
    try:
        if operation == "add":
            return add_complex(complex(args[0]), complex(args[1]))
        
        elif operation == "subtract":
            return subtract_complex(complex(args[0]), complex(args[1]))
        
        elif operation == "multiply":
            return multiply_complex(complex(args[0]), complex(args[1]))
        
        elif operation == "divide":
            return divide_complex(complex(args[0]), complex(args[1]))
        
        elif operation == "solve":
            return solve_complex_equation(args[0])
        
        elif operation == "root":
            return nth_root_complex(complex(args[0]), int(args[1]))
        
        else:
            return "Operation not supported."
        
    except Exception as e:
        return str(e)

Creating tool for complex operations

In [ ]:
complex_algebra_tool = Tool(
    name = "Complex Algebra",
    func = perform_complex_operation,
    description = "Performs complex algebraic operations like addition, subtraction, multiplication, division, solving equations and finding roots."
)

Since perform_complex_operation cannot directly solve the problem, I need to parse the arguments

In [ ]:
import re

def parse_question(question: str):
    # Match for operations and complex numbers
    match = re.match(r"(add|subtract|multiply|divide) the complex numbers \(([-+]?\d*\.?\d+[+-]\d*\.?\d*j)\) and \(([-+]?\d*\.?\d+[+-]\d*\.?\d*j)\)", question, re.IGNORECASE)
    
    if match:
        operation = match.group(1).lower()
        num1 = match.group(2)
        num2 = match.group(3)
        return perform_complex_operation(operation, num1, num2)
    else:
        return "Operation not supported or unable to parse the question."

Creating tool for parsing

In [ ]:
parse_algebra_tool = Tool(
    name = "Parse Algebra",
    func = parse_question,  # Use the parsing function here
    description = "Performs parsing complex algebraic operations like addition, subtraction, multiplication, and division."
)

Initialization of agent relying on our tools

In [ ]:
tools = [parse_algebra_tool, complex_algebra_tool]

react = initialize_agent(tools, llm, AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [ ]:
question = "add the complex numbers (3+4j) and (1-2j)."
response = react.run(question)
print(response)